In [28]:
# import libraries
import pandas as pd
import numpy as np
import math

### Read raw data and aggregate

In [66]:
# read data
datapath='/Users/soni6/github/soccer-analytics/data/archive/Datasets'
# final_dataset=pd.read_csv(datapath+'/final_dataset.csv')
stats = ['FTHG','FTAG','FTR','HTHG','HTAG','HTR','HS','AS','HST','AST',
                                       'HC','AC','HF','AF','HY','AY','HR','AR']
attributes = ['Date','HomeTeam','AwayTeam']
all_seasons_data=pd.DataFrame(columns=attributes+stats)
# seasons 2010-2020
for i in range(10,20):
    dataseason=pd.read_csv(datapath+'/20{}-{}.csv'.format(i,i+1))
    dataseason=dataseason[dataseason['Div']=='E0']
    all_seasons_data = pd.concat([all_seasons_data,dataseason], axis=0, join='inner',ignore_index=True)
#seasons 2000-2009
for i in range(0,9):
    dataseason=pd.read_csv(datapath+'/200{}-0{}.csv'.format(i,i+1))
    dataseason=dataseason[dataseason['Div']=='E0']
    all_seasons_data = pd.concat([all_seasons_data,dataseason], axis=0, join='inner',ignore_index=True)
#season 2009-2010
dataseason=pd.read_csv(datapath+'/2009-10.csv')
dataseason=dataseason[dataseason['Div']=='E0']
all_seasons_data = pd.concat([all_seasons_data,dataseason], axis=0, join='inner',ignore_index=True)
all_seasons_data['Date']= pd.to_datetime(all_seasons_data['Date']) 
#season rankings
standings = pd.read_csv(datapath+'/EPLStandings.csv')
# standings=standings.fillna(30)

### Columns Description
Date = Match Date (dd/mm/yy)\
HomeTeam = Home Team\
AwayTeam = Away Team\
FTHG = Full Time Home Team Goals\
FTAG = Full Time Away Team Goals\
FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)\
HTHG = Half Time Home Team Goals\
HTAG = Half Time Away Team Goals\
HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)\
HS = Home Team Shots\
AS = Away Team Shots\
HST = Home Team Shots on Target\
AST = Away Team Shots on Target\
HHW = Home Team Hit Woodwork\
AHW = Away Team Hit Woodwork\
HC = Home Team Corners\
AC = Away Team Corners\
HF = Home Team Fouls Committed\
AF = Away Team Fouls Committed\
HY = Home Team Yellow Cards\
AY = Away Team Yellow Cards\
HR = Home Team Red Cards\
AR = Away Team Red Cards\

In [67]:
# check for null values in each column
print('Column','  No. of Null Vals')
all_seasons_data.isnull().sum()

Column   No. of Null Vals


Date        0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
HTHG        0
HTAG        0
HTR         0
HS          0
AS          0
HST         0
AST         0
HC          0
AC          0
HF          0
AF          0
HY          0
AY          0
HR          0
AR          0
dtype: int64

### Feature Engineering
Next we create some important feature in order to capture useful information about team's performance in recent matches.

In [68]:
teams=all_seasons_data['HomeTeam'].unique()
print(teams)
print("Number of teams=", teams.size)

['Aston Villa' 'Blackburn' 'Bolton' 'Chelsea' 'Sunderland' 'Tottenham'
 'Wigan' 'Wolves' 'Liverpool' 'Man United' 'Arsenal' 'Birmingham'
 'Everton' 'Stoke' 'West Brom' 'West Ham' 'Fulham' 'Newcastle' 'Man City'
 'Blackpool' 'QPR' 'Swansea' 'Norwich' 'Reading' 'Southampton'
 'Crystal Palace' 'Hull' 'Cardiff' 'Leicester' 'Burnley' 'Bournemouth'
 'Watford' 'Middlesbrough' 'Brighton' 'Huddersfield' 'Sheffield United'
 'Charlton' 'Coventry' 'Derby' 'Leeds' 'Bradford' 'Ipswich' 'Middlesboro'
 'Portsmouth']
Number of teams= 44


We observe that there are 44 unique teams. Although, there are 20 teams which play the league each year, each season few teams move to relegation zone and are disqualified from playing the league and some new teams enter the league.

Next we create some useful insights using the existing features. In particular we create following features.
* Performance in last 3 home matches of home team
* Performance in last 3 away matches of away team
* Overall head to head performance
* Standing of home and away team in previous seasons


In [90]:
# some helper functions which will be used later
# all_seasons_data and standings required
def get_previous_match_stats(team: str,date: pd.datetime,num_prev_matches:int,venue:str):
    """
    home or away specific
    :param team-Name of team
    :param date-
    :param num_prev_matches-
    :param venue-Home or Away
    Eg. team_data=get_previous_match_stats('Leeds',pd.to_datetime('2001-01-01'),5,'Home')
    """
    team_data = all_seasons_data[all_seasons_data[venue+'Team']==team]
    team_data = team_data.sort_values(by='Date')
    team_data = team_data [team_data['Date']<date]
    team_data = team_data.tail(num_prev_matches)
    team_data = team_data.iloc[:,3:]
    return team_data.mean(axis=0,skipna=True)

def get_previous_encouters_stats(x):
    """
    get average of all previous meets on the given venue
    """
    homeTeam=x.HomeTeam
    awayTeam=x.AwayTeam
    prev_encounters = all_seasons_data[(all_seasons_data['HomeTeam']== homeTeam) & (all_seasons_data['AwayTeam']== awayTeam) ]
    default=True
    w,d,l=0.5,0.5,0.5
    results = prev_encounters['FTR'].value_counts()
    if 1 in list(results.index):
        w=results[1]
        default = False
    if 0 in list(results.index):
        d=results[0]
        default = False
    if -1 in list(results.index):
        l=results[-1]
        default = False
    if default==True:
        net_chances=0
    else:
        net_chances = (1*w+0*d-1*l)/results.shape[0]
    return net_chances

def get_last_season_year(x):
    """
    :param x: row of dataframe
    season year for season 2009-2010 is 2010
    """
    x['Date']= pd.to_datetime(x['Date']) 
    month = x['Date'].month
    year = x['Date'].year
    last_season = year
    if 7 < month <= 12:
        last_year = year
    else:
        last_year = year-1
    return last_year

def get_previous_year_position(x, home=True):
    last_year =x.LastYear
    try:
        if home==True:
            last_season_position = standings[standings['Team']==x['HomeTeam']].iloc[0][str(int(last_year))]
        if home==False:
            last_season_position = standings[standings['Team']==x['AwayTeam']].iloc[0][str(int(last_year))]
    except:
        # position not available in table as team did not play in last league
        pass
    return last_season_position

def result_to_score(x,time='full'):
    """
    Convert Win,Draw, Loss to numeric features
    W: +1, Home team win
    L: -1, Home temam lost
    D: 0, Match draw
    :param: time= full or half
    """
    if time=='full':
        if x['FTR']=='H':
            score = 1
        elif x['FTR']=='A':
            score=-1
        else:
            score=0
    if time=='half':
        if x['HTR']=='H':
            score = 1
        elif x['HTR']=='A':
            score=-1
        else:
            score=0
    return score

def get_form_factor(x,team, num_matches=5):
    """
    get recent form irrespective of home or away
    :param x- row of dataframe
    :param num_matches- number of matches to consider to calculate form, default 10
    :param team: home or away
    return: current form of the team over last
    """
    if team == "home":
        team = x.HomeTeam
    elif team == "away":
        team = x.AwayTeam
    else:
        raise Exception("Incorrect team specified: should be home or away")
    x['Date']= pd.to_datetime(x['Date']) 
    team_data_home = all_seasons_data[all_seasons_data['HomeTeam']==team]
    team_data_away = all_seasons_data[all_seasons_data['AwayTeam']==team]
    team_data = pd.concat([team_data_home,team_data_away])
    team_data = team_data.sort_values(by='Date')
    team_data = team_data [team_data['Date']<x['Date']]
    team_data = team_data.tail(num_matches)
    alpha = 0.6
    # calculate exponential form function
    form = 0
    if team_data.shape[0]>0:
        for i in range(min(team_data.shape[0],num_matches)):
            row = team_data.iloc[i]
            if team == row.HomeTeam:
                shots = row['HST']
            if team == row.AwayTeam:
                shots = row['AST']   
            form = form + shots*math.e ** (-1 * (alpha * i)) 
    return form

### Convert match results to labels and get average performance of home and away teams over last 3 matches at home and away venues respectively

In [70]:
# convert results to labels
all_seasons_data['FTR']=all_seasons_data.apply(lambda x: result_to_score(x,time='full'),axis=1)
all_seasons_data['HTR']=all_seasons_data.apply(lambda x: result_to_score(x,time='half'),axis=1)

In [71]:
# get average performance over 3 matches
get_previous_stats = True
if get_previous_stats==True:
    all_seasons_data['Date']= pd.to_datetime(all_seasons_data['Date']) 
    num_prev_matches = 3
    p_1 = [stat + "-H" for stat in stats] #Home match stats
    p_4 = [stat + "-A" for stat in stats] #away match stats
    previous_stats_data=pd.DataFrame(index=np.arange(0, all_seasons_data.shape[0]),columns=attributes+['FTR']+p_1+p_4)
    error_index=[]
    for index,row in all_seasons_data.iterrows():
        prev_home_data=get_previous_match_stats(row.HomeTeam,row.Date,num_prev_matches,'Home')
        prev_away_data=get_previous_match_stats(row.AwayTeam,row.Date,num_prev_matches,'Away')
        try:
            previous_stats_data.iloc[index] = [row.Date,row.HomeTeam,row.AwayTeam,row['FTR']] \
            + list(prev_home_data)+ list(prev_away_data)
        except:
            # there aren't enough matches to look back into past
            error_index.append(index)
    

### Get recent form
We use shots on targets over the last 5 matches as the measure of form

In [72]:
previous_stats_data['HomeForm'] = previous_stats_data.apply(lambda x:get_form_factor(x,team='home'),axis=1 )
previous_stats_data['AwayForm'] = previous_stats_data.apply(lambda x:get_form_factor(x,team='away'),axis=1 )

### Get previous encounters

Assign a score on scale of -1 to 1 which accounts for all previous meetings on the given venue

In [73]:
# get a probability of home team winning or away team winning
previous_stats_data['PreviousEncounters']=previous_stats_data.apply(lambda x: get_previous_encouters_stats(x),axis=1)

### Merge Previous seasons standings if available


In [74]:
# get previous year
previous_stats_data['LastYear']=previous_stats_data.apply(lambda x: get_last_season_year(x),axis=1)
previous_stats_data =previous_stats_data[previous_stats_data['LastYear']<=2016]

In [79]:
# get previous standings

previous_stats_data['HomePreviousPosition']=previous_stats_data.apply(lambda x: get_previous_year_position(x,home=True),axis=1)
previous_stats_data['AwayPreviousPosition']=previous_stats_data.apply(lambda x: get_previous_year_position(x,home=False),axis=1)
# if position not found, assign a position of 30
previous_stats_data['HomePreviousPosition']=previous_stats_data['HomePreviousPosition'].fillna(30)
previous_stats_data['AwayPreviousPosition']=previous_stats_data['AwayPreviousPosition'].fillna(30)

In [ ]:
# check for null values in each column
print('Column','  No. of Null Vals')
previous_stats_data.isnull().sum()


In [94]:
# drop nan rows
X=previous_stats_data.dropna()
print(list(X.columns))

['Date', 'HomeTeam', 'AwayTeam', 'FTR', 'FTHG-H', 'FTAG-H', 'FTR-H', 'HTHG-H', 'HTAG-H', 'HTR-H', 'HS-H', 'AS-H', 'HST-H', 'AST-H', 'HC-H', 'AC-H', 'HF-H', 'AF-H', 'HY-H', 'AY-H', 'HR-H', 'AR-H', 'FTHG-A', 'FTAG-A', 'FTR-A', 'HTHG-A', 'HTAG-A', 'HTR-A', 'HS-A', 'AS-A', 'HST-A', 'AST-A', 'HC-A', 'AC-A', 'HF-A', 'AF-A', 'HY-A', 'AY-A', 'HR-A', 'AR-A', 'HomeForm', 'AwayForm', 'PreviousEncounters', 'LastYear', 'HomePreviousPosition', 'AwayPreviousPosition']


In [95]:
finalData=X.drop(columns=['Date','HomeTeam','AwayTeam','LastYear'])
finalData.to_csv(datapath+'/finalData.csv',index=False)

$Form=\sum_{i=1}^5Shots*(\exp^{-\alpha*i})$


Rob meeting: earn parameters for the form. treat it like. trianing problem
Compare with betting sites